In [21]:
from __future__ import print_function
import os

import numpy as np
import tensorflow as tf

from keras.layers import Dense, LSTM, Embedding, Input, RepeatVector, Lambda
from keras.models import Model, load_model
from keras.layers.merge import concatenate
from data_generator import DataGenerator

In [58]:
class ModelConfig():
    def __init__(self,
                 input_path,
                 embedding_path,
                 embedding_dim=100,
                 batch_size=32,
                 epochs=20,
                 min_score=0,
                 max_nb_words=50000,
                 max_nb_examples=None,
                 max_sequence_length=300,
                 lstm_size_encoder=256,
                 lstm_size_decoder=256,
                 intermediate_size=64,
                 latent_size=16,
                 kl_weight=1,
                 optimizer="adam",
                 validation_split=0.1):
        self.input_path = input_path
        self.embedding_path = embedding_path
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size
        self.epochs = epochs
        self.min_score = min_score
        self.max_nb_words = max_nb_words
        self.max_nb_examples = max_nb_examples
        self.max_sequence_length = max_sequence_length
        self.lstm_size_encoder = lstm_size_encoder
        self.lstm_size_decoder = lstm_size_decoder
        self.intermediate_size = intermediate_size
        self.latent_size = latent_size
        self.kl_weight = kl_weight
        self.optimizer = optimizer
        self.validation_split = validation_split

In [94]:
class UncondDecodeLstmCvae(object):
    """Class to hold LSTM CVAE model
    This implementation users the repeated code z as the only input to the decoder 
    (i.e. unconditional decoder)
    """
    def __init__(self, model_config, tokenizer):
        self.config = model_config
        self.config.word_index = tokenizer.word_index
        self.config.num_words = min(model_config.max_nb_words, 
                                    len(tokenizer.word_index))
    def load_embedding(self):
        """Load and prepare embedding matrix"""
        embeddings_index = {}
        f = open(self.config.embedding_path)
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        f.close()
        embedding_matrix = np.zeros((self.config.num_words + 1, self.config.embedding_dim))
        for word, i in self.config.word_index.items():
            if i >= self.config.max_nb_words:
                continue
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector
        return embedding_matrix
    
    def build(self):
        """Construct lstm cvae model"""
        # Load embedding in Embedding layer
        embedding_matrix = self.load_embedding()
        embedding_layer = Embedding(self.config.num_words + 1,
                                    self.config.embedding_dim,
                                    weights=[embedding_matrix],
                                    input_length=self.config.max_sequence_length,
                                    trainable=False)
        
        # Q(z|X,y) -- encoder
        # embedded sequence input
        sequence_inputs = Input(batch_shape=(self.config.batch_size, self.config.max_sequence_length), dtype='int32')
        embedded_inputs = embedding_layer(sequence_inputs)
        x = LSTM(self.config.lstm_size_encoder, return_sequences=False)(embedded_inputs)
        score_inputs = Input(batch_shape=(self.config.batch_size, 1))
        x_joint = concatenate([x, score_inputs], axis=1)
        x_encoded = Dense(self.config.intermediate_size, activation='relu')(x_joint)
        z_mean = Dense(self.config.latent_size)(x_encoded)
        z_log_sigma = Dense(self.config.latent_size)(x_encoded)

        # Sample z ~ Q(z|X,y)
        def sampling(args):
            z_mean, z_log_sigma = args
            epsilon = tf.random_normal(shape=(self.config.batch_size, self.config.latent_size), 
                                       mean=0., stddev=1.)
            return z_mean + tf.exp(z_log_sigma/2.) * epsilon
        z = Lambda(sampling)([z_mean, z_log_sigma])
        z_cond = concatenate([z, score_inputs], axis=1)

        # P(X|z,y) -- decoder
        z_repeated = RepeatVector(self.config.max_sequence_length)(z_cond)
        
        decoder_h = LSTM(self.config.lstm_size_decoder, return_sequences=True)
        decoder_out = Dense(self.config.num_words + 1)
        
        h_decoded = decoder_h(z_repeated)
        x_decoded = decoder_out(h_decoded)
        # Construct three models
        # vae
        vae = Model([sequence_inputs, score_inputs], x_decoded)
        # encoder
        encoder = Model([sequence_inputs, score_inputs], z_mean)
        # generator
        generator_z_inputs = Input(batch_shape=(self.config.batch_size, self.config.latent_size))
        generator_z_cond = concatenate([generator_z_inputs, score_inputs], axis=1)
        generator_z_repeated = RepeatVector(self.config.max_sequence_length)(generator_z_cond)
        generator_h_decoded = decoder_h(generator_z_repeated)
        generator_x_decoded = decoder_out(generator_h_decoded)
        generator = Model([generator_z_inputs, score_inputs], generator_x_decoded)
        
        def vae_loss(y_true, y_pred):
            """ Calculate loss = reconstruction loss + KL loss for each data in minibatch """
            # E[log P(X|z,y)]
            recon = tf.reduce_mean(
                tf.contrib.keras.backend.sparse_categorical_crossentropy(
                output=y_pred, target=y_true, from_logits=True),
                axis=1)
            # D_KL(Q(z|X,y) || P(z|X)); calculate in closed form as both dist. are Gaussian
            kl = - 0.5 * tf.reduce_mean(1 + z_log_sigma - tf.square(z_mean) - tf.exp(z_log_sigma), axis=-1)
            return recon + kl * self.config.kl_weight
        
        vae.compile(loss=vae_loss, optimizer=self.config.optimizer)
        
        self.vae = vae
        self.encoder = encoder
        self.encoder = encoder
        
    def fit(self, x, y_s):
        x_reshaped = x.reshape(x.shape[0], x.shape[1], 1)
        self.vae.fit([x, y_s], x_reshaped,
                     batch_size=self.config.batch_size,
                     epochs=self.config.epochs)

In [71]:
# Test
BASE_DIR = os.getcwd().replace("/lstm_cvae", "")
GLOVE_DIR = BASE_DIR.replace("/code", "/glove.6B/")
embedding_path=os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')
TEXT_DATA_DIR = os.path.join(BASE_DIR.replace("/code", '/joke-dataset/'), "reddit_jokes.json")

print(TEXT_DATA_DIR)
print(embedding_path)

model_config = ModelConfig(input_path=TEXT_DATA_DIR,
                           embedding_path=embedding_path,
                           max_nb_examples=100)

/Users/tongwang/Playground/deepjoke/joke-dataset/reddit_jokes.json
/Users/tongwang/Playground/deepjoke/glove.6B/glove.6B.100d.txt


In [6]:
data_generator = DataGenerator(
    input_path=model_config.input_path,
    min_score=model_config.min_score, 
    max_nb_words=model_config.max_nb_words,
    max_nb_examples=model_config.max_nb_examples,
    max_sequence_length=model_config.max_sequence_length,
    validation_split=model_config.validation_split)

x_train, y_l_train, y_s_train, x_val, y_l_val, y_s_val, tokenizer = data_generator.generate()

//anaconda/lib/python2.7/site-packages/unidecode/__init__.py:50: RuntimeWarning: Surrogate character u'\ud83c' will be ignored. You might be using a narrow Python build.
  return _unidecode(string)
//anaconda/lib/python2.7/site-packages/unidecode/__init__.py:50: RuntimeWarning: Surrogate character u'\udde9' will be ignored. You might be using a narrow Python build.
  return _unidecode(string)
//anaconda/lib/python2.7/site-packages/unidecode/__init__.py:50: RuntimeWarning: Surrogate character u'\uddf0' will be ignored. You might be using a narrow Python build.
  return _unidecode(string)
//anaconda/lib/python2.7/site-packages/unidecode/__init__.py:50: RuntimeWarning: Surrogate character u'\ud83d' will be ignored. You might be using a narrow Python build.
  return _unidecode(string)
//anaconda/lib/python2.7/site-packages/unidecode/__init__.py:50: RuntimeWarning: Surrogate character u'\ude02' will be ignored. You might be using a narrow Python build.
  return _unidecode(string)
//anaconda

Read in 194553 jokes.
Read in 194553 scores.

Found 1332 unique words; using 1332 unique words

Shape of training features: (90, 300)
Shape of training language model labels: (90, 300, 1)
Shape of training score labels: (90,)
Shape of validation features: (10, 300)
Shape of validation language model labels: (10, 300, 1)
Shape of validation score labels: (10,)


In [95]:
cvae = UncondDecodeLstmCvae(model_config, tokenizer)

In [96]:
cvae.build()

TODO:
* Get rid of batch_shape
* slice random var to be of the same shape as batch

In [97]:
cvae.fit(x_train, y_s_train)

Epoch 1/20
64/90 [====================>.........] - ETA: 2s - loss: 8.3082

ValueError: Cannot feed value of shape (26, 300) for Tensor u'input_56:0', which has shape '(32, 300)'